<a href="https://www.kaggle.com/code/handandegerli/rule-based-classification-n-lead-yield-calculation?scriptVersionId=178937581" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

### Rule-Based Classification & Lead Yield Calculation

In [1]:
import pandas as pd

In [2]:
df = pd.read_excel("/kaggle/input/gezinomi/miuul_gezinomi.xlsx")

In [3]:
df.head()

,SaleId,SaleDate,CheckInDate,Price,ConceptName,SaleCityName,CInDay,SaleCheckInDayDiff,Seasons
0,415122,2022-12-03,2022-12-03,79.304029,Herşey Dahil,Antalya,Saturday,0,Low
1,415103,2022-12-03,2022-12-03,45.970696,Yarım Pansiyon,Antalya,Saturday,0,Low
2,404034,2022-09-12,2022-09-13,77.838828,Herşey Dahil,Antalya,Tuesday,1,High
3,415094,2022-12-03,2022-12-10,222.710623,Yarım Pansiyon,İzmir,Saturday,7,Low
4,414951,2022-12-01,2022-12-03,140.476190,Yarım Pansiyon,İzmir,Saturday,2,Low


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59164 entries, 0 to 59163
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   SaleId              59164 non-null  int64         
 1   SaleDate            59164 non-null  datetime64[ns]
 2   CheckInDate         59164 non-null  datetime64[ns]
 3   Price               59151 non-null  float64       
 4   ConceptName         59164 non-null  object        
 5   SaleCityName        59164 non-null  object        
 6   CInDay              59164 non-null  object        
 7   SaleCheckInDayDiff  59164 non-null  int64         
 8   Seasons             59164 non-null  object        
dtypes: datetime64[ns](2), float64(1), int64(2), object(4)
memory usage: 4.1+ MB


**Question: How many unique cities are there? What are their frequencies?**

In [5]:
df["SaleCityName"].nunique()

6

In [6]:
df["SaleCityName"].value_counts()

SaleCityName
Antalya    31649
Muğla      10662
Aydın      10646
Diğer       3245
İzmir       2507
Girne        455
Name: count, dtype: int64

**Question: How many unique concepts are there?**

In [7]:
df["ConceptName"].nunique()

3

**Question: How many sales were made from which Concept?**

In [8]:
df.groupby("ConceptName")["ConceptName"].count()

ConceptName
Herşey Dahil      53186
Oda + Kahvaltı     2419
Yarım Pansiyon     3559
Name: ConceptName, dtype: int64

In [9]:
df.groupby("ConceptName").agg({"SaleId": "count"})

,SaleId
ConceptName,
Herşey Dahil,53186
Oda + Kahvaltı,2419
Yarım Pansiyon,3559


**Question: How much was earned from sales in total by city?**

In [10]:
df.groupby("SaleCityName").agg({"Price": "sum"})

,Price
SaleCityName,
Antalya,2.041911e+06
Aydın,5.732960e+05
Diğer,1.545723e+05
Girne,2.706503e+04
Muğla,6.658422e+05
İzmir,1.659348e+05


**Question: How much was earned according to Concept Types?**

In [11]:
df.groupby("ConceptName").agg({"Price": "sum"})

,Price
ConceptName,
Herşey Dahil,3.332911e+06
Oda + Kahvaltı,1.213084e+05
Yarım Pansiyon,1.744024e+05


**Question: Sort the price averages by city in decreasing order.**

In [12]:
df.groupby("SaleCityName").agg({"Price": "mean"}).sort_values(by="Price", ascending = False)

,Price
SaleCityName,
İzmir,66.267905
Antalya,64.521474
Muğla,62.461746
Girne,59.483579
Aydın,53.855896
Diğer,47.707497


**Question: What is the Price average according to concepts?**

In [13]:
df.groupby("ConceptName").agg({"Price": "mean"})

,Price
ConceptName,
Herşey Dahil,62.672260
Oda + Kahvaltı,50.252010
Yarım Pansiyon,49.030742


**Question: Convert the SaleCheckInDayDiff variable into a categorical variable.**

In [14]:
bins = [0, 7, 30, 90, df["SaleCheckInDayDiff"].max()]
labels = ["Last_Minuters", "Potential_Planners", "Planners", "Early_Bookers"]

In [15]:
bins, labels

([0, 7, 30, 90, 630],
 ['Last_Minuters', 'Potential_Planners', 'Planners', 'Early_Bookers'])

In [16]:
df["CheckInType"] = pd.cut(df["SaleCheckInDayDiff"], bins = bins, labels=labels, include_lowest = True)

In [17]:
df.tail()

,SaleId,SaleDate,CheckInDate,Price,ConceptName,SaleCityName,CInDay,SaleCheckInDayDiff,Seasons,CheckInType
59159,51817,2016-01-05,2016-10-10,54.304636,Herşey Dahil,Antalya,Monday,279,Low,Early_Bookers
59160,51816,2016-01-05,2016-10-10,54.304636,Herşey Dahil,Antalya,Monday,279,Low,Early_Bookers
59161,51814,2016-01-05,2016-01-06,40.562914,Herşey Dahil,Diğer,Wednesday,1,Low,Last_Minuters
59162,51736,2016-01-04,2016-01-05,69.847682,Yarım Pansiyon,Diğer,Tuesday,1,Low,Last_Minuters
59163,51731,2016-01-04,2016-08-22,158.940397,Herşey Dahil,Antalya,Monday,231,High,Early_Bookers


**Question: What are the average earnings in SaleCityName, ConceptName, CheckInType breakdown?**

In [18]:
df.groupby(["SaleCityName", "ConceptName", "CheckInType"], observed=True).agg({"Price": ["mean","count"]})

Price       
                                                     mean  count
SaleCityName ConceptName    CheckInType                         
Antalya      Herşey Dahil   Last_Minuters       62.752916  14148
                            Potential_Planners  64.903700   8874
                            Planners            67.879960   4490
                            Early_Bookers       66.494885   3281
             Oda + Kahvaltı Last_Minuters       65.353740    503
...                                                   ...    ...
İzmir        Oda + Kahvaltı Early_Bookers       66.928111      3
             Yarım Pansiyon Last_Minuters       57.788126    636
                            Potential_Planners  64.804043    198
                            Planners            60.507887     47
                            Early_Bookers       65.905583     14

[72 rows x 2 columns]

**Question: Sort the output of the "SaleCityName", "ConceptName", "Seasons" breakdown according to price.**

In [19]:
agg_df = df.groupby(["SaleCityName", "ConceptName", "Seasons"]).agg({"Price": "mean"}).sort_values("Price", ascending=False)

**Question: Convert the names in the index into variable names.**

In [20]:
agg_df.reset_index(inplace=True)

**Questions:
#Define new level-based sales and add them to the data set as a variable.
#Name of the new variable to be added: sales_level_based
#You need to create the sales_level_based variable by combining the observations in the output you obtained in the previous question.**

In [21]:
agg_df.head()

,SaleCityName,ConceptName,Seasons,Price
0,Girne,Herşey Dahil,High,103.935448
1,Girne,Herşey Dahil,Low,90.935944
2,İzmir,Yarım Pansiyon,High,87.657303
3,Diğer,Herşey Dahil,Low,87.310882
4,Diğer,Herşey Dahil,High,83.787273


In [22]:
df["sales_level_based"] = df[["SaleCityName", "ConceptName", "Seasons"]].apply(lambda x:"_".join(x), axis= 1)
df["sales_level_based"] = df["sales_level_based"].str.upper()
df["sales_level_based"]

0          ANTALYA_HERŞEY DAHIL_LOW
1        ANTALYA_YARIM PANSIYON_LOW
2         ANTALYA_HERŞEY DAHIL_HIGH
3          İZMIR_YARIM PANSIYON_LOW
4          İZMIR_YARIM PANSIYON_LOW
                    ...            
59159      ANTALYA_HERŞEY DAHIL_LOW
59160      ANTALYA_HERŞEY DAHIL_LOW
59161        DIĞER_HERŞEY DAHIL_LOW
59162      DIĞER_YARIM PANSIYON_LOW
59163     ANTALYA_HERŞEY DAHIL_HIGH
Name: sales_level_based, Length: 59164, dtype: object

In [23]:
agg_df = df[["sales_level_based", "Price"]]
agg_df.head()

,sales_level_based,Price
0,ANTALYA_HERŞEY DAHIL_LOW,79.304029
1,ANTALYA_YARIM PANSIYON_LOW,45.970696
2,ANTALYA_HERŞEY DAHIL_HIGH,77.838828
3,İZMIR_YARIM PANSIYON_LOW,222.710623
4,İZMIR_YARIM PANSIYON_LOW,140.476190


**Question: Divide new customers (personas) into segments.
• Divide new personas into 4 segments according to Price.
• Add the segments to agg_df as variables with the name Segment.
• Describe the segments (Group by segments and get price mean, max, sum).**

In [24]:
agg_df["Segment"] = pd.qcut(agg_df["Price"], 4, labels = ["D", "C", "B", "A"])

/tmp/ipykernel_18/1218002253.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agg_df["Segment"] = pd.qcut(agg_df["Price"], 4, labels = ["D", "C", "B", "A"])


In [25]:
agg_df.head()

,sales_level_based,Price,Segment
0,ANTALYA_HERŞEY DAHIL_LOW,79.304029,A
1,ANTALYA_YARIM PANSIYON_LOW,45.970696,C
2,ANTALYA_HERŞEY DAHIL_HIGH,77.838828,B
3,İZMIR_YARIM PANSIYON_LOW,222.710623,A
4,İZMIR_YARIM PANSIYON_LOW,140.476190,A


In [26]:
agg_df.groupby("Segment").agg({"Price":["mean", "max", "sum"]})

/tmp/ipykernel_18/3034291487.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  agg_df.groupby("Segment").agg({"Price":["mean", "max", "sum"]})


Price                           
               mean          max           sum
Segment                                       
D         25.659182    35.343035  3.794480e+05
C         44.597904    54.261294  6.596030e+05
B         64.975656    77.962578  9.607300e+05
A        110.153543  4880.471380  1.628840e+06

**Questions: Classify new customers and estimate how much revenue they can bring.
• How much income on average is expected for a person who wants to have an all-inclusive and high-season holiday in Antalya?
• In which segment will a holidaymaker who goes to a half-board hotel in Girne in low season be included?**

In [27]:
agg_df["sales_level_based"] = agg_df["sales_level_based"].str.upper()

/tmp/ipykernel_18/1715259595.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agg_df["sales_level_based"] = agg_df["sales_level_based"].str.upper()


In [28]:
new_user = 'ANTALYA_HERŞEY DAHIL_HIGH'
agg_df[agg_df['sales_level_based'] == new_user]

,sales_level_based,Price,Segment
2,ANTALYA_HERŞEY DAHIL_HIGH,77.838828,B
76,ANTALYA_HERŞEY DAHIL_HIGH,101.594424,A
79,ANTALYA_HERŞEY DAHIL_HIGH,70.970696,B
80,ANTALYA_HERŞEY DAHIL_HIGH,109.383378,A
85,ANTALYA_HERŞEY DAHIL_HIGH,107.238606,A
...,...,...,...
59139,ANTALYA_HERŞEY DAHIL_HIGH,69.081126,B
59140,ANTALYA_HERŞEY DAHIL_HIGH,46.054084,C
59145,ANTALYA_HERŞEY DAHIL_HIGH,41.757174,C
59146,ANTALYA_HERŞEY DAHIL_HIGH,41.757174,C


In [29]:
new_user2 = 'GIRNE_YARIM PANSIYON_LOW'
agg_df[agg_df['sales_level_based'] == new_user2]

,sales_level_based,Price,Segment
671,GIRNE_YARIM PANSIYON_LOW,101.343101,A
18623,GIRNE_YARIM PANSIYON_LOW,42.328042,C
21093,GIRNE_YARIM PANSIYON_LOW,50.671551,C
21094,GIRNE_YARIM PANSIYON_LOW,46.448921,C
42331,GIRNE_YARIM PANSIYON_LOW,32.876712,D
...,...,...,...
59125,GIRNE_YARIM PANSIYON_LOW,111.754967,A
59126,GIRNE_YARIM PANSIYON_LOW,111.754967,A
59136,GIRNE_YARIM PANSIYON_LOW,69.536424,B
59142,GIRNE_YARIM PANSIYON_LOW,36.423841,C


In [30]:
new_user3 = 'İZMIR_HERŞEY DAHIL_HIGH'
agg_df[agg_df['sales_level_based'] == new_user3]

,sales_level_based,Price,Segment
1901,İZMIR_HERŞEY DAHIL_HIGH,137.362637,A
1967,İZMIR_HERŞEY DAHIL_HIGH,23.943135,D
1968,İZMIR_HERŞEY DAHIL_HIGH,23.943135,D
2021,İZMIR_HERŞEY DAHIL_HIGH,91.575092,A
2118,İZMIR_HERŞEY DAHIL_HIGH,90.163309,A
...,...,...,...
58864,İZMIR_HERŞEY DAHIL_HIGH,56.291391,B
58869,İZMIR_HERŞEY DAHIL_HIGH,55.408389,B
58878,İZMIR_HERŞEY DAHIL_HIGH,76.986755,B
58960,İZMIR_HERŞEY DAHIL_HIGH,83.245033,A
